In [44]:
import pandas as pd
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn import set_config
from time import time
from datetime import datetime
from dateutil import parser
from sklearn.model_selection import cross_validate
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder

In [180]:
def prep_df(csv):
    global df_original
    df_original = pd.read_csv(csv)
    df = pd.read_csv(csv)
    df["date_today"] = datetime.now()
    df = df.dropna(subset = ["original_estimate_complete_datetime"])
    df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
    df = df.assign(time_since_estimate =  df['date_today'] - df['estimate_date_parse'])
    df["days_since_estimate"] = df["time_since_estimate"].apply(lambda x: int(str(x).split()[0]))
   
    del df["time_since_estimate"]
    #df = df[df["days_since_estimate"] >= 90]
    df1 = df[df["suppCnt"] == 0]
    df2 = df[df["suppCnt"] != 0]
    df1["suppInd"] = 0
    df2["suppInd"] = 1
    df = pd.concat([df1,df2], axis = 0)
    df["paid_loss_amount"] = df["paid_loss_amount"].apply(lambda x: abs(x))
    df["primary_point_of_impact"].fillna("Unknown",inplace = True)
    df["total_loss_points"].fillna(999,inplace = True)
    df["insured_fault_pct"].fillna(999,inplace = True)
    global df_w_days
    df_w_days = df
    df = df[["claim_number","company","coverage_grouping","detailed_loss_cause",'driveable_ind',"insured_fault_pct","insured_fault_rating","line_of_business","loss_cause","loss_party_type","loss_state","original_estimate_complete_datetime","payAfterCloseInd","policy_state","policy_type","primary_point_of_impact","saved_vehicle_as_type","secondary_point_of_impact","total_loss_points","vehicle_make","vehicle_year","suppInd","maxNet","minNet","maxFile","minFile","days_since_estimate"]]
    return df

In [181]:
time1 = time()
df = prep_df("Supp Prediction.csv")

target_name = "suppInd"
Y = df[target_name]
X = df.drop(columns=[target_name,"payAfterCloseInd","days_since_estimate"])

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_29780\4095150450.py:3: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_29780\4095150450.py:4: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_29780\4095150450.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_29780\4095150450.py:15: SettingWithCopyWarning: 
A value is trying to be set on 

In [182]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

In [183]:
numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

In [184]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

In [185]:
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])

In [186]:
model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))

In [187]:
data_train, data_test, target_train, target_test = train_test_split(
    X, Y, random_state=42)

In [188]:
_ = model.fit(data_train, target_train)

C:\Users\TWILLI16\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [189]:
model.score(data_test, target_test)

0.9411715387625063

In [193]:
df_under_90 = df[df["days_since_estimate"] < 90][df["suppInd"] == 0]
results = model.predict(df[df["days_since_estimate"] < 90][df["suppInd"] == 0])
time2 = time()
print(time2 - time1)

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_29780\3746934050.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_under_90 = df[df["days_since_estimate"] < 90][df["suppInd"] == 0]
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_29780\3746934050.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results = model.predict(df[df["days_since_estimate"] < 90][df["suppInd"] == 0])


566.3798363208771


In [194]:
df_under_90["prediction"] = results
df_under_90[df_under_90["suppInd"] == 0][df_under_90["prediction"] == 1]

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_29780\1375738392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_under_90["prediction"] = results


,claim_number,company,coverage_grouping,detailed_loss_cause,driveable_ind,insured_fault_pct,insured_fault_rating,line_of_business,loss_cause,loss_party_type,...,total_loss_points,vehicle_make,vehicle_year,suppInd,maxNet,minNet,maxFile,minFile,days_since_estimate,prediction


In [113]:
len(df[df["suppInd"] == 1]) / len(df)

0.4702520479991616

In [192]:
df

,claim_number,company,coverage_grouping,detailed_loss_cause,driveable_ind,insured_fault_pct,insured_fault_rating,line_of_business,loss_cause,loss_party_type,...,secondary_point_of_impact,total_loss_points,vehicle_make,vehicle_year,suppInd,maxNet,minNet,maxFile,minFile,days_since_estimate
0,01-004-642006,Connect,Collision,Collision While Turning,No,0.0,Claimant Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,NaN,40.0,Toyota,2017,0,-500.00,-500.00,0.00,0.00,102
1,01-004-717683,Connect,Collision,Insured Rear-ended Claimant,No,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,Right Front Corner,59.0,Mercury,2005,0,4256.30,4256.30,4756.30,4756.30,63
2,01-003-423937,Amfam,Collision,Collision While Turning,No,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,NaN,40.0,Jeep,2016,0,11137.15,11137.15,11637.15,11637.15,433
3,01-004-251708,MSA,Comprehensive,Front End Collision,No,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,NaN,34.0,Ford Truck,2020,0,31123.23,31123.23,31123.23,31123.23,216
4,01-004-281333,MSA,Collision,Insured Rear-ended Claimant,No,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,NaN,45.0,Ford,2013,0,7084.24,7084.24,7584.24,7584.24,204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286610,01-004-720923,Amfam,Comprehensive,NaN,Yes,0.0,No fault,Personal Auto Line,Hail,Insured's loss,...,NaN,999.0,Tesla,2022,1,802.62,201.25,1302.62,701.25,83
286612,01-004-695162,Amfam,Comprehensive,NaN,Yes,0.0,No fault,Personal Auto Line,Hail,Insured's loss,...,NaN,999.0,Chevrolet Truck,2021,1,2028.30,1384.90,2528.30,1884.90,89
286613,01-002-469312,Amfam,Collision,Sideswipe,No,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,NaN,26.0,Hyundai,2017,1,7441.78,6652.11,7941.78,7152.11,750
286615,01-002-491095,Amfam,Collision,Insured Rear-ended Claimant,Yes,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,NaN,21.0,Suzuki,2008,1,1650.80,595.71,2150.80,1095.71,771
